In [1]:
import spacy
import random
import datetime
import json
import math
import dateutil.parser
from pathlib import Path
from spacy.util import minibatch, compounding

largenlp = spacy.load('en_core_web_lg')

### JSON Content utlities for the TMDB corpus

In [2]:
#Iterate through the movies
def rawTmdbMovies(filename):
    return json.load(open(filename))


def writeTmdmMovies(rawMoviesJson, path):
    with open(path, 'w') as f:
        json.dump(rawMoviesJson, f)

def tmdbMovies(filename="../tmdb.json"):
    tmdbMovies = rawTmdbMovies(filename)
    for movieId, tmdbMovie in tmdbMovies.items():
        yield (movieId, tmdbMovie)

def indexableMovies(filename="../tmdb.json"):
    """ Generates TMDB movies, similar to how ES Bulk indexing
        uses a generator to generate bulk index/update actions """
    for movieId, tmdbMovie in tmdbMovies(filename):
        try:
            releaseDate = None
            if 'release_date' in tmdbMovie and len(tmdbMovie['release_date']) > 0:
                releaseDate = tmdbMovie['release_date'] + 'T00:00:00Z'

            yield {'id': movieId,
                   'title': tmdbMovie['title'],
                   'overview': tmdbMovie['overview'],
                   'tagline': tmdbMovie['tagline'],
                   'directors': [director['name'] for director in tmdbMovie['directors']],
                   'cast': [castMember['name'] for castMember in tmdbMovie['cast']],
                   'genres': [genre['name'] for genre in tmdbMovie['genres']],
                   'release_date': releaseDate,
                   'vote_average': tmdbMovie['vote_average'] if 'vote_average' in tmdbMovie else None,
                   'vote_count': int(tmdbMovie['vote_count']) if 'vote_count' in tmdbMovie else None,
                   'location_entities': tmdbMovie['location_entities'] if 'location_entities' in tmdbMovie else [],
                   'location': tmdbMovie['location'] if 'location' in tmdbMovie else [],
                   'location_city': tmdbMovie['location_city'] if 'location_city' in tmdbMovie else [],
                   'location_state': tmdbMovie['location_state'] if 'location_state' in tmdbMovie else [],
                   'location_country': tmdbMovie['location_country'] if 'location_country' in tmdbMovie else [],
                   }
        except KeyError as k: # Ignore any movies missing these attributes
            continue

### Get the overview texts

We're going to use the overview text to train lowercase entities for our queries.

This is free training data!  We'll take the entities from the true-cased overview text, and apply them to the lower-cased text, and train a new model

In [3]:
def overviews(filename="../tmdb.json"):
    #Get all the overviews
    texts = []
    for movie_id, movie in tmdbMovies(filename=filename):
        if "overview" in movie and isinstance(movie["overview"], str) and len(movie["overview"]):
            texts.append(movie["overview"])
    return texts

In [4]:
overview_texts = overviews()

In [27]:
#For all the overviews in all the movies, find their entities, and test if they are found when lowercased
def get_ent_info(span):
    return [ent.text+'_'+str(ent[0].i)+'_'+str(ent[0].i+len(ent.text))+'_'+ent.label_ for ent in span.ents]
def show_lowercase_entity_differences(texts):
    for doc in largenlp.pipe(texts):
        ents_orig = get_ent_info(doc)
        lowertext = largenlp(doc.text.lower())
        ents_lower = get_ent_info(lowertext)
        if len(ents_orig) != len(ents_lower):
            print(doc.text)
            print(ents_orig,ents_lower)
        print('-----------------------------')
show_lowercase_entity_differences(overview_texts[0:100])        

This feature-length special consists of three interwoven stories. In a mysterious and remote snowy outpost, Matt and Potter share a Christmas meal, swapping creepy tales of their earlier lives in the outside world. Matt is a charismatic American trying to bring the reserved, secretive Potter out of his shell. But are both men who they appear to be? A woman gets thrust into a nightmarish world of 'smart' gadgetry. Plus a look at what would happen if you could 'block' people in real life.
['three_7_12_CARDINAL', 'Matt_19_23_PERSON', 'Christmas_24_33_DATE', 'Matt_39_43_PERSON', 'American_43_51_NORP'] ['three_7_12_CARDINAL', 'american_43_51_NORP']
-----------------------------
Raj is a rich, carefree, happy-go-lucky second generation NRI. Simran is the daughter of Chaudhary Baldev Singh, who in spite of being an NRI is very strict about adherence to Indian values. Simran has left for India to be married to her childhood fiancé. Raj leaves for India with a mission at his hands, to claim his

In the civil wars of 16th century Japan, two ambitious peasants in Nakanogo want to make their fortunes. The potter Genjuro intends to sell his wares for vast profits in the local city, Omizo, while his brother-in-law Tobei wishes to become a samurai. A respected sage warns them against seeking profit in times of war but his advice goes unheeded, and their insistency leads to the ruin of their families.
['16th century_5_17_DATE', 'Japan_7_12_GPE', 'two_9_12_CARDINAL', 'Nakanogo_13_21_GPE', 'Genjuro_22_29_PERSON', 'Omizo_36_41_ORG', 'Tobei_45_50_PERSON'] ['16th century_5_17_DATE', 'japan_7_12_GPE', 'two_9_12_CARDINAL']
-----------------------------
Peace in 17th-century Japan causes the Shogunate's breakup of warrior clans, throwing thousands of samurai out of work and into poverty. An honorable end to such fate under the samurai code is ritual suicide, or hara-kiri.
['17th-century_2_14_DATE', 'Japan_5_10_GPE', 'Shogunate_8_17_DATE', 'thousands_16_25_CARDINAL'] ['17th-century_2_14_DATE'

The Pianist is a film adapted from the biography of Wladyslaw Szpilman. A Jewish-Polish pianist who during the second world war lived and hid miraculously in Warsaw after having gone through a terrible tragedy. A film from Roman Polanski.
['Wladyslaw Szpilman_10_28_PERSON', 'Jewish_14_20_NORP', 'second_21_27_ORDINAL', 'Warsaw_29_35_GPE', 'Roman Polanski_41_55_PERSON'] ['second_21_27_ORDINAL', 'warsaw_29_35_GPE']
-----------------------------
-----------------------------
-----------------------------
A young boy comes of age in the most peculiar of circumstances. Tonight is the very first time his Papa and Grandpa are taking him to work. In an old wooden boat they row far out to sea, and with no land in sight, they stop and wait. A big surprise awaits the boy as he discovers his family's most unusual line of work. Should he follow the example of his Papa, or his Grandpa? Will he be able to find his own way in the midst of their conflicting opinions and timeworn traditions?
['Tonight_13

In [43]:
#For all the overviews in all the movies, find their entities, and assign them to the lowercase text.
#...this takes about 5 mins to run
def get_ents(span):
    return [(ent[0].i,ent[0].i+len(ent.text),ent.label_) for ent in span.ents]
def make_training_data(texts):
    training_data = []
    for doc in largenlp.pipe(texts):
        ents_orig = get_ents(doc)
        lowertext = doc.text.lower()
        training_data.append((lowertext, {"entities":ents_orig}))
    return training_data
training_data = make_training_data(overview_texts[0:1000])

In [44]:
training_data[0:100]

[("this feature-length special consists of three interwoven stories. in a mysterious and remote snowy outpost, matt and potter share a christmas meal, swapping creepy tales of their earlier lives in the outside world. matt is a charismatic american trying to bring the reserved, secretive potter out of his shell. but are both men who they appear to be? a woman gets thrust into a nightmarish world of 'smart' gadgetry. plus a look at what would happen if you could 'block' people in real life.",
  {'entities': [(7, 12, 'CARDINAL'),
    (19, 23, 'PERSON'),
    (24, 33, 'DATE'),
    (39, 43, 'PERSON'),
    (43, 51, 'NORP')]}),
 ('raj is a rich, carefree, happy-go-lucky second generation nri. simran is the daughter of chaudhary baldev singh, who in spite of being an nri is very strict about adherence to indian values. simran has left for india to be married to her childhood fiancé. raj leaves for india with a mission at his hands, to claim his lady love under the noses of her whole family. th

In [10]:
#Train the model, which takes about 10 minutes
def train_new_model(training_data):
    nlp = spacy.blank("en")
    optimizer = nlp.begin_training()
    for i in range(20):
        random.shuffle(training_data)
        for text, annotations in training_data:
            nlp.update([text], [annotations], sgd=optimizer)
    nlp.to_disk("tmdb_toy_query_model")

In [11]:
train_new_model(training_data)

### Testing our new model
Load the model from disk that we created, and see if it can recognize lowercase entities (it doesn't, why not?):

In [12]:
querynlp = spacy.load('./tmdb_toy_query_model')
querytext = "boxing revenge in moscow"
print('Large model:',largenlp(querytext).ents)
print('New model:',querynlp(querytext).ents)

Large model: (moscow,)
New model: ()


### Transfer learning

We need to continue training using the existing *en_core_web_lg* model, so that we don't lose everything it's already learned!

In [49]:
def train_existing_model(training_data):

    model="en_core_web_lg"
    output_dir="./tmdb_query_model2"
    n_iter=20    
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)

    #Get the ner pipe so we can add labels
    ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in training_data:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    # test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    for text, _ in training_data:
        doc = nlp2(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [50]:
train_existing_model(training_data)

Loaded model 'en_core_web_lg'
Losses {'ner': 4062.5211378877084}
Losses {'ner': 3520.2656382336836}
Losses {'ner': 3408.378206396671}
Losses {'ner': 3382.5737820367126}
Losses {'ner': 3327.1573684431337}
Losses {'ner': 3425.119708511764}
Losses {'ner': 3361.507634419297}
Losses {'ner': 3431.3407483645046}
Losses {'ner': 3426.356060959462}
Losses {'ner': 3316.582214677932}
Losses {'ner': 3315.914994667683}
Losses {'ner': 3282.8826127473603}
Losses {'ner': 3340.69822059452}
Losses {'ner': 3366.8762019902338}
Losses {'ner': 3419.417801472113}
Losses {'ner': 3257.64147815929}
Losses {'ner': 3337.172249523892}
Losses {'ner': 3239.314986646036}
Losses {'ner': 3224.4778961376824}
Losses {'ner': 3305.7948111412516}
Entities [('christmas', 'DATE')]
Tokens [('christmas', 'DATE', 3), ('in', '', 2), ('tokyo', '', 2), (',', '', 2), ('japan', '', 2), ('.', '', 2), ('three', '', 2), ('homeless', '', 2), ('friends', '', 2), (':', '', 2), ('a', '', 2), ('young', '', 2), ('girl', '', 2), (',', '', 2), (

Entities []
Tokens [('spirited', '', 2), ('away', '', 2), ('is', '', 2), ('an', '', 2), ('oscar', '', 2), ('winning', '', 2), ('japanese', '', 2), ('animated', '', 2), ('film', '', 2), ('about', '', 2), ('a', '', 2), ('ten', '', 2), ('year', '', 2), ('old', '', 2), ('girl', '', 2), ('who', '', 2), ('wanders', '', 2), ('away', '', 2), ('from', '', 2), ('her', '', 2), ('parents', '', 2), ('along', '', 2), ('a', '', 2), ('path', '', 2), ('that', '', 2), ('leads', '', 2), ('to', '', 2), ('a', '', 2), ('world', '', 2), ('ruled', '', 2), ('by', '', 2), ('strange', '', 2), ('and', '', 2), ('unusual', '', 2), ('monster', '', 2), ('-', '', 2), ('like', '', 2), ('animals', '', 2), ('.', '', 2), ('her', '', 2), ('parents', '', 2), ('have', '', 2), ('been', '', 2), ('changed', '', 2), ('into', '', 2), ('pigs', '', 2), ('along', '', 2), ('with', '', 2), ('others', '', 2), ('inside', '', 2), ('a', '', 2), ('bathhouse', '', 2), ('full', '', 2), ('of', '', 2), ('these', '', 2), ('creatures', '', 2), (

Entities []
Tokens [('the', '', 2), ('story', '', 2), ('of', '', 2), ('the', '', 2), ('last', '', 2), ('seal', '', 2), ('child', '', 2), ('’s', '', 2), ('journey', '', 2), ('home', '', 2), ('.', '', 2), ('after', '', 2), ('their', '', 2), ('mother', '', 2), ('’s', '', 2), ('disappearance', '', 2), (',', '', 2), ('ben', '', 2), ('and', '', 2), ('saoirse', '', 2), ('are', '', 2), ('sent', '', 2), ('to', '', 2), ('live', '', 2), ('with', '', 2), ('granny', '', 2), ('in', '', 2), ('the', '', 2), ('city', '', 2), ('.', '', 2), ('when', '', 2), ('they', '', 2), ('resolve', '', 2), ('to', '', 2), ('return', '', 2), ('to', '', 2), ('their', '', 2), ('home', '', 2), ('by', '', 2), ('the', '', 2), ('sea', '', 2), (',', '', 2), ('their', '', 2), ('journey', '', 2), ('becomes', '', 2), ('a', '', 2), ('race', '', 2), ('against', '', 2), ('time', '', 2), ('as', '', 2), ('they', '', 2), ('are', '', 2), ('drawn', '', 2), ('into', '', 2), ('a', '', 2), ('world', '', 2), ('ben', '', 2), ('knows', '', 2)

Entities []
Tokens [('private', '', 2), ('eye', '', 2), ('jake', '', 2), ('gittes', '', 2), ('lives', '', 2), ('off', '', 2), ('the', '', 2), ('murky', '', 2), ('moral', '', 2), ('climate', '', 2), ('of', '', 2), ('sunbaked', '', 2), (',', '', 2), ('pre', '', 2), ('-', '', 2), ('world', '', 2), ('war', '', 2), ('ii', '', 2), ('southern', '', 2), ('california', '', 2), ('.', '', 2), ('hired', '', 2), ('by', '', 2), ('a', '', 2), ('beautiful', '', 2), ('socialite', '', 2), ('to', '', 2), ('investigate', '', 2), ('her', '', 2), ('husband', '', 2), ("'s", '', 2), ('extra', '', 2), ('-', '', 2), ('marital', '', 2), ('affair', '', 2), (',', '', 2), ('gittes', '', 2), ('is', '', 2), ('swept', '', 2), ('into', '', 2), ('a', '', 2), ('maelstrom', '', 2), ('of', '', 2), ('double', '', 2), ('dealings', '', 2), ('and', '', 2), ('deadly', '', 2), ('deceits', '', 2), (',', '', 2), ('uncovering', '', 2), ('a', '', 2), ('web', '', 2), ('of', '', 2), ('personal', '', 2), ('and', '', 2), ('political', '

Entities []
Tokens [('a', '', 2), ('rat', '', 2), ('named', '', 2), ('remy', '', 2), ('dreams', '', 2), ('of', '', 2), ('becoming', '', 2), ('a', '', 2), ('great', '', 2), ('french', '', 2), ('chef', '', 2), ('despite', '', 2), ('his', '', 2), ('family', '', 2), ("'s", '', 2), ('wishes', '', 2), ('and', '', 2), ('the', '', 2), ('obvious', '', 2), ('problem', '', 2), ('of', '', 2), ('being', '', 2), ('a', '', 2), ('rat', '', 2), ('in', '', 2), ('a', '', 2), ('decidedly', '', 2), ('rodent', '', 2), ('-', '', 2), ('phobic', '', 2), ('profession', '', 2), ('.', '', 2), ('when', '', 2), ('fate', '', 2), ('places', '', 2), ('remy', '', 2), ('in', '', 2), ('the', '', 2), ('sewers', '', 2), ('of', '', 2), ('paris', '', 2), (',', '', 2), ('he', '', 2), ('finds', '', 2), ('himself', '', 2), ('ideally', '', 2), ('situated', '', 2), ('beneath', '', 2), ('a', '', 2), ('restaurant', '', 2), ('made', '', 2), ('famous', '', 2), ('by', '', 2), ('his', '', 2), ('culinary', '', 2), ('hero', '', 2), (',',

...

Entities []
Tokens [('fictional', '', 2), ('documentary', '', 2), ('about', '', 2), ('the', '', 2), ('life', '', 2), ('of', '', 2), ('human', '', 2), ('chameleon', '', 2), ('leonard', '', 2), ('zelig', '', 2), (',', '', 2), ('a', '', 2), ('man', '', 2), ('who', '', 2), ('becomes', '', 2), ('a', '', 2), ('celebrity', '', 2), ('in', '', 2), ('the', '', 2), ('1920s', '', 2), ('due', '', 2), ('to', '', 2), ('his', '', 2), ('ability', '', 2), ('to', '', 2), ('look', '', 2), ('and', '', 2), ('act', '', 2), ('like', '', 2), ('whoever', '', 2), ('is', '', 2), ('around', '', 2), ('him', '', 2), ('.', '', 2), ('clever', '', 2), ('editing', '', 2), ('places', '', 2), ('zelig', '', 2), ('in', '', 2), ('real', '', 2), ('newsreel', '', 2), ('footage', '', 2), ('of', '', 2), ('woodrow', '', 2), ('wilson', '', 2), (',', '', 2), ('babe', '', 2), ('ruth', '', 2), (',', '', 2), ('and', '', 2), ('others', '', 2), ('.', '', 2)]
Entities []
Tokens [('when', '', 2), ('a', '', 2), ('machine', '', 2), ('that',

Entities [('u.s.', 'ORG')]
Tokens [('u.s', 'ORG', 3), ('.', 'ORG', 1), ('navy', '', 2), ('seal', '', 2), ('chris', '', 2), ('kyle', '', 2), ('takes', '', 2), ('his', '', 2), ('sole', '', 2), ('mission', '', 2), ('—', '', 2), ('protect', '', 2), ('his', '', 2), ('comrades', '', 2), ('—', '', 2), ('to', '', 2), ('heart', '', 2), ('and', '', 2), ('becomes', '', 2), ('one', '', 2), ('of', '', 2), ('the', '', 2), ('most', '', 2), ('lethal', '', 2), ('snipers', '', 2), ('in', '', 2), ('american', '', 2), ('history', '', 2), ('.', '', 2), ('his', '', 2), ('pinpoint', '', 2), ('accuracy', '', 2), ('not', '', 2), ('only', '', 2), ('saves', '', 2), ('countless', '', 2), ('lives', '', 2), ('but', '', 2), ('also', '', 2), ('makes', '', 2), ('him', '', 2), ('a', '', 2), ('prime', '', 2), ('target', '', 2), ('of', '', 2), ('insurgents', '', 2), ('.', '', 2), ('despite', '', 2), ('grave', '', 2), ('danger', '', 2), ('and', '', 2), ('his', '', 2), ('struggle', '', 2), ('to', '', 2), ('be', '', 2), ('a

Entities []
Tokens [('the', '', 2), ('true', '', 2), ('story', '', 2), ('of', '', 2), ('christopher', '', 2), ('gardner', '', 2), (',', '', 2), ('who', '', 2), ('invests', '', 2), ('heavily', '', 2), ('in', '', 2), ('a', '', 2), ('device', '', 2), ('known', '', 2), ('as', '', 2), ('a', '', 2), ("'", '', 2), ('bone', '', 2), ('density', '', 2), ('scanner', '', 2), ("'", '', 2), (',', '', 2), ('only', '', 2), ('to', '', 2), ('find', '', 2), ('himself', '', 2), ('struggle', '', 2), ('to', '', 2), ('sell', '', 2), ('the', '', 2), ('product', '', 2), ('as', '', 2), ('it', '', 2), ("'s", '', 2), ('just', '', 2), ('marginally', '', 2), ('better', '', 2), ('than', '', 2), ('the', '', 2), ('current', '', 2), ('technology', '', 2), (',', '', 2), ('and', '', 2), ('much', '', 2), ('more', '', 2), ('expensive', '', 2), ('.', '', 2), ('his', '', 2), ('wife', '', 2), ('leaves', '', 2), ('him', '', 2), (',', '', 2), (' ', '', 2), ('he', '', 2), ('loses', '', 2), ('his', '', 2), ('house', '', 2), (',',

Entities [('two', 'CARDINAL')]
Tokens [('two', 'CARDINAL', 3), ('soldiers', '', 2), ('from', '', 2), ('opposite', '', 2), ('sites', '', 2), ('get', '', 2), ('stuck', '', 2), ('between', '', 2), ('the', '', 2), ('front', '', 2), ('lines', '', 2), ('in', '', 2), ('the', '', 2), ('same', '', 2), ('trench', '', 2), ('.', '', 2), ('the', '', 2), ('un', '', 2), ('is', '', 2), ('asked', '', 2), ('to', '', 2), ('free', '', 2), ('them', '', 2), ('and', '', 2), ('both', '', 2), ('sides', '', 2), ('agree', '', 2), ('on', '', 2), ('a', '', 2), ('ceasefire', '', 2), (',', '', 2), ('but', '', 2), ('will', '', 2), ('they', '', 2), ('stick', '', 2), ('to', '', 2), ('it', '', 2), ('?', '', 2)]
Entities [('robin hood', 'PERSON')]
Tokens [('robin', 'PERSON', 3), ('hood', 'PERSON', 1), ('(', '', 2), ('errol', '', 2), ('flynn', '', 2), (')', '', 2), ('fights', '', 2), ('nobly', '', 2), ('for', '', 2), ('justice', '', 2), ('against', '', 2), ('the', '', 2), ('evil', '', 2), ('sir', '', 2), ('guy', '', 2), (

Entities [('hubert', 'PERSON')]
Tokens [('hubert', 'PERSON', 3), (',', '', 2), ('a', '', 2), ('brash', '', 2), ('17-year', '', 2), ('-', '', 2), ('old', '', 2), (',', '', 2), ('is', '', 2), ('confused', '', 2), ('and', '', 2), ('torn', '', 2), ('by', '', 2), ('a', '', 2), ('love', '', 2), ('-', '', 2), ('hate', '', 2), ('relationship', '', 2), ('with', '', 2), ('his', '', 2), ('mother', '', 2), ('that', '', 2), ('consumes', '', 2), ('him', '', 2), ('more', '', 2), ('and', '', 2), ('more', '', 2), ('each', '', 2), ('day', '', 2), ('.', '', 2), ('after', '', 2), ('distressing', '', 2), ('ordeals', '', 2), ('and', '', 2), ('tragic', '', 2), ('episodes', '', 2), (',', '', 2), ('hubert', '', 2), ('will', '', 2), ('find', '', 2), ('his', '', 2), ('mother', '', 2), ('on', '', 2), ('the', '', 2), ('banks', '', 2), ('of', '', 2), ('saint', '', 2), ('lawrence', '', 2), ('river', '', 2), (',', '', 2), ('where', '', 2), ('he', '', 2), ('grew', '', 2), ('up', '', 2), (',', '', 2), ('and', '', 2), (

In [46]:
with open('training_data.json', 'w') as fp:
    json.dump(training_data[:1], fp)
with open('dev_data.json', 'w') as fp:
    json.dump(training_data[2:3], fp)

In [45]:
training_data[2:3]

[('framed in the 1940s for the double murder of his wife and her lover, upstanding banker andy dufresne begins a new life at the shawshank prison, where he puts his accounting skills to work for an amoral warden. during his long stretch in prison, dufresne comes to be admired by the other inmates -- including an older prisoner named red -- for his integrity and unquenchable sense of hope.',
  {'entities': [(2, 11, 'DATE'),
    (17, 30, 'PERSON'),
    (25, 34, 'ORG'),
    (48, 56, 'PERSON')]})]

In [48]:
TD = [("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]})]
DD = [("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]})]
with open('training_data.json', 'w') as fp:
    json.dump(TD, fp)
with open('dev_data.json', 'w') as fp:
    json.dump(DD, fp)